# Implementation of DeepFace using Keras
In this notebook a [Keras implementation](https://github.com/swghosh/DeepFace) from Github of Facebook's [DeepFace](https://research.fb.com/publications/deepface-closing-the-gap-to-human-level-performance-in-face-verification/) is loaded and used. The weights are trained on the publicly available VGG dataset

In [9]:
import keras
import numpy as np
from os import path
from os import listdir
from os.path import isfile, join
from PIL import Image 
from keras.models import Model
import tensorflow
import csv
import pandas as pd

In [10]:
# implement model architecture

IMAGE_SIZE = (152, 152) # set by the model 
CHANNELS = 3 # RGB image
NUM_CLASSES = 8631 # classification layer will be removed 
LEARN_RATE = 0.01
MOMENTUM = 0.9

def create_classifying_deepface(image_size=IMAGE_SIZE, channels=CHANNELS, num_classes=NUM_CLASSES, learn_rate=LEARN_RATE, momentum=MOMENTUM):
    """
    Deep CNN architecture primarily for Face Recognition,
    Face Verification and Face Representation (feature extraction) purposes
    "DeepFace: Closing the Gap to Human-Level Performance in Face Verification"
    CNN architecture proposed by Taigman et al. (CVPR 2014)
    """

    wt_init = keras.initializers.RandomNormal(mean=0, stddev=0.01)
    bias_init = keras.initializers.Constant(value=0.5)

    """
    Construct certain functions 
    for using some common parameters
    with network layers
    """
    def conv2d_layer(**args):
        return keras.layers.Conv2D(**args, 
            kernel_initializer=wt_init, 
            bias_initializer=bias_init,
            activation=keras.activations.relu)
    def lc2d_layer(**args):
        return keras.layers.LocallyConnected2D(**args, 
            kernel_initializer=wt_init, 
            bias_initializer=bias_init,
            activation=keras.activations.relu)
    def dense_layer(**args):
        return keras.layers.Dense(**args, 
            kernel_initializer=wt_init, 
            bias_initializer=bias_init)

    """
    Create the network using
    tf.keras.layers.Layer(s)
    """
    deepface = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=(*image_size, channels), name='I0'),
        conv2d_layer(filters=32, kernel_size=11, name='C1'),
        keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same',  name='M2'),
        conv2d_layer(filters=16, kernel_size=9, name='C3'),
        lc2d_layer(filters=16, kernel_size=9, name='L4'),
        lc2d_layer(filters=16, kernel_size=7, strides=2, name='L5'),
        lc2d_layer(filters=16, kernel_size=5, name='L6'),
        keras.layers.Flatten(name='F0'),
        dense_layer(units=4096, activation=keras.activations.relu, name='F7'),
        keras.layers.Dropout(rate=0.5, name='D0'),
        dense_layer(units=num_classes, activation=keras.activations.softmax, name='F8')
    ], name='DeepFace')
    # deepface.summary()

    """
    A tf.keras.optimizers.SGD will
    be used for training,
    and compile the model
    """
    sgd_opt = keras.optimizers.SGD(lr=learn_rate, momentum=momentum)
    cce_loss = keras.losses.categorical_crossentropy

    deepface.compile(optimizer=sgd_opt, loss=cce_loss, metrics=['accuracy'])
    
    return deepface


In [11]:
DOWNLOAD_PATH = 'https://github.com/swghosh/DeepFace/releases/download/weights-vggface2-2d-aligned/VGGFace2_DeepFace_weights_val-0.9034.h5.zip'
MD5_HASH = '0b21fb70cd6901c96c19ac14c9ea8b89'

In [12]:
def get_weights():
    filename = 'deepface.zip'
    downloaded_file_path = keras.utils.get_file(filename, DOWNLOAD_PATH, 
        md5_hash=MD5_HASH, extract=True)
    downloaded_h5_file = path.join(path.dirname(downloaded_file_path), 
        path.basename(DOWNLOAD_PATH).rstrip('.zip'))
    return downloaded_h5_file

In [13]:
def create_deepface():
    model = create_classifying_deepface()
    weights = get_weights()
    model.load_weights(weights)
    model2 = Model(model.input, model.layers[-2].output)
    model2.summary()
    return model2

In [14]:
model = create_deepface()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
I0 (InputLayer)              (None, 152, 152, 3)       0         
_________________________________________________________________
C1 (Conv2D)                  (None, 142, 142, 32)      11648     
_________________________________________________________________
M2 (MaxPooling2D)            (None, 71, 71, 32)        0         
_________________________________________________________________
C3 (Conv2D)                  (None, 63, 63, 16)        41488     
_________________________________________________________________
L4 (LocallyConnected2D)      (None, 55, 55, 16)        62774800  
_________________________________________________________________
L5 (LocallyConnected2D)      (None, 25, 25, 16)        7850000   
_________________________________________________________________
L6 (LocallyConnected2D)      (None, 21, 21, 16)        2829

## How to put an image through the model 
The image needs to be resized to the above defined size to be able to put through the model.

In [15]:
data_dir = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien\kdv"
age_ranges = [[1, 3], [4, 16], [17, 40]]
kdv_rep = []
ID_rep = []

    
for [low_age, high_age] in age_ranges:
    
    # open directories
    kdv_dir = data_dir + "\kdv-patients-age-group-"+str(low_age) + "-" + str(high_age)
    ID_dir = data_dir + "\kdv-selected-ID-controls-age-group-"+str(low_age) + "-" + str(high_age)

    # get list of filenames
    files_kdv = [f for f in listdir(kdv_dir) if (isfile(join(kdv_dir, f)) & ("crop_sized.jpg" in f))]
    files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f)) & ("crop_sized.JPG" in f))]

#     print(len(files_kdv))
#     print(len(files_ID))

    # for each kdv image save deepface rep as list:
    for filename in files_kdv:
        im = Image.open(join(kdv_dir, filename))
        output = model.predict(np.expand_dims(im, axis=0))
        kdv_rep.append([filename] + output[0].tolist())  
    

    # for each ID image save deepface rep as list:
    for filename in files_ID:
        im = Image.open(join(ID_dir, filename))
        output = model.predict(np.expand_dims(im, axis=0))
        ID_rep.append([filename] + output[0].tolist())



In [16]:
print(len(kdv_rep))
print(len(ID_rep))

70
70


In [19]:
# location to save representation
csv_file_kdv = data_dir+ "\\representations\kdv-patients-deepface.csv"
csv_file_ID = data_dir+ "\\representations\ID-controls-deepface.csv"

# save representation of kdv patients
with open(csv_file_kdv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(kdv_rep)

# save representation of ID controls
with open(csv_file_ID, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(ID_rep)

print("Done with saving all deepface representations.")

Done with saving all deepface representations.
